In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction\\research'

In [3]:
os.chdir('c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction')

In [4]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    train_data_path: str
    test_data_path: str


In [6]:
from src.gemstonePricePrediction.constants import *
from src.gemstonePricePrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        #   root_dir=Path(config.root_dir),
          source_URL=config.source_URL,
          local_data_file=Path(config.local_data_file),
          unzip_dir=Path(config.unzip_dir),
          train_data_path=config.train_data_path,
          test_data_path=config.test_data_path
         )


        return data_ingestion_config

In [ ]:
import os
import sys
import logging
import zipfile
import pandas as pd

from urllib import request
from pathlib import Path
from sklearn.model_selection import train_test_split

from gemstonePricePrediction.entity.config_entity import DataIngestionConfig
from gemstonePricePrediction.exception import CustomException
from gemstonePricePrediction.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:

        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )

                logging.info(
                    f"File: {filename} downloaded with following info: \n{headers}"
                )
            else:
                logging.info(
                    f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
                )

        except Exception as e:
            raise CustomException(e, sys)


    def extract_zip_file(self) -> None:

        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logging.info("Zip file extracted successfully")

        except Exception as e:
            raise CustomException(e, sys)


    # Data ingestion without raw data saving
    def initiate_data_ingestion(self):

        try:
            logging.info("Starting data ingestion process")

            # Read extracted dataset
            data_file_path = os.path.join(self.config.unzip_dir, "data.csv")

            df = pd.read_csv(data_file_path)

            logging.info("Train Test Split Initiated")

            # Train Test Split
            train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

            # Ensure directory exists
            os.makedirs(os.path.dirname(self.config.train_data_path), exist_ok=True)

            # Save train and test sets
            train_set.to_csv(self.config.train_data_path, index=False, header=True)
            test_set.to_csv(self.config.test_data_path, index=False, header=True)

            logging.info("Ingestion of Data is completed")

            return (
                self.config.train_data_path,
                self.config.test_data_path
            )

        except Exception as e:
            raise CustomException(e, sys)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

    train_path, test_path = data_ingestion.initiate_data_ingestion()

    print("Train file created at:", train_path)
    print("Test file created at:", test_path)

except Exception as e:
    raise e


[2026-02-11 16:09:17,826: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\config\config.yaml]
[2026-02-11 16:09:17,841: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\params.yaml]
[2026-02-11 16:09:17,847: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\schema.yaml]
[2026-02-11 16:09:17,849: INFO: common: created directory at: artifacts]
[2026-02-11 16:09:17,854: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-11 16:09:25,021: INFO: 897280208: File: artifacts\data_ingestion\gemstone.zip downloaded with following info: 
Connection: close
Content-Length: 16739931
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0faceee890ef6b94e489da9f8d4cc5de02f91ddde0239579ea019970a1df2006"
